In [1]:
import numpy as np
import pandas as pd
import imblearn                                      
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

In [2]:
data = pd.read_csv('initial_preprocessing_reg.csv')
data = data[data['DAS28-CRP 3m'].notnull()]    ### We don't need unknown category for method-1


In [3]:
data

,grp,Type I IFN activity,IFNβ activity final,IFNα activity final,IFNβ/α ratio final,pres_imuran,pres_minocin,nonpresNSAIDs_use,NSAIDs_use,tender_jts_28,...,tender_jts_28_BL,swollen_jts_28_BL,md_global_assess_BL,pt_global_assess_BL,di_BL,pt_pain_BL,usresultsCRP_BL,usresultsIgA_BL,usresultsIgG_BL,usresultsIgM_BL
0,0,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,5,3,35.0,10.0,0.00,10.0,4.41,284.0,1640.0,94.0
1,1,0.814954,0.714954,0.714954,1.000000,0.0,0.0,1.0,NaN,1.0,...,3,3,35.0,17.0,0.25,25.0,1.19,207.0,1010.0,56.0
2,1,4.582632,2.869955,4.482632,0.640239,0.0,0.0,0.0,0.0,5.0,...,2,6,40.0,10.0,0.00,5.0,0.40,190.0,657.0,100.0
3,1,0.015811,NaN,NaN,NaN,0.0,0.0,NaN,NaN,14.0,...,3,9,55.0,50.0,0.25,60.0,1.00,44.0,948.0,51.0
4,2,0.683486,0.583486,0.583486,1.000000,0.0,0.0,NaN,NaN,2.0,...,14,13,65.0,20.0,0.25,65.0,0.25,155.0,824.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2,0.379131,0.279131,0.279131,1.000000,0.0,0.0,1.0,NaN,0.0,...,0,9,70.0,15.0,0.00,65.0,6.02,125.0,1000.0,77.0
292,1,0.000000,NaN,NaN,NaN,0.0,0.0,1.0,NaN,2.0,...,8,7,55.0,80.0,1.00,80.0,7.33,353.0,859.0,211.0
293,2,0.000000,NaN,NaN,NaN,0.0,0.0,1.0,NaN,2.0,...,2,6,35.0,50.0,0.00,15.0,22.36,266.0,1330.0,253.0
294,2,0.000000,NaN,NaN,NaN,0.0,0.0,1.0,NaN,0.0,...,5,5,35.0,15.0,0.00,15.0,0.92,163.0,1500.0,44.0


In [4]:
rs = 1337
X = data.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y set
y = data['DAS28-CRP 3m']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1,random_state = rs)

<ipython-input-4-6aac00e6f16e>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = data.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y set


In [7]:
imputer = KNNImputer(n_neighbors=15)                     ### Imputing the null values using KNN with neighbors as 15
fit_train = imputer.fit(X_train)
imput_train = fit_train.transform(X_train)
c = list(X_train.columns)
imput_train_df = pd.DataFrame(imput_train, columns = c)

In [8]:
imput_test = fit_train.transform(X_test)                 ### Imputing the test set using the training set
c = list(X_test.columns)
imput_test_df = pd.DataFrame(imput_test, columns = c)

In [10]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in imput_train_df.columns:
    if i not in labels:
        mean = imput_train_df[i].mean()
        std = imput_train_df[i].std()
        imput_train_df[i] = (imput_train_df[i] - mean) / std
        imput_test_df[i] = (imput_test_df[i] - mean) / std

In [11]:
### Remove "hxhtn" feature here if the code is not running
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    imput_train_df = imput_train_df.drop(i,1)
    imput_test_df = imput_test_df.drop(i,1)

<ipython-input-11-649ae34e813e>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_train_df = imput_train_df.drop(i,1)
<ipython-input-11-649ae34e813e>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_test_df = imput_test_df.drop(i,1)


In [12]:
Y_train1 = Y_train.reset_index()
Y_train1 = Y_train1.drop(columns=['index'])

In [14]:
Y_train1 = Y_train.reset_index()
Y_train1 = Y_train1.drop(columns=['index'])
imput_train_df['y'] = Y_train1
imput_train_df.to_csv('train_method1_rg.csv', index = False)

In [15]:
Y_test1 = Y_test.reset_index()
Y_test1 = Y_test1.drop(columns=['index'])
imput_test_df['y'] = Y_test1 
imput_test_df.to_csv('test_method1_rg.csv', index = False)